In [1]:
# 加载模块
import requests
from pytz import timezone
from datetime import datetime

from vnpy.trader.constant import Exchange, Interval
from vnpy.trader.object import BarData

In [2]:
# 开始K线查询函数
CHINA_TZ = timezone("Asia/Shanghai")

def download_binance_minute_data(symbol: str, start: str, end: str):
    """基于代码和交易所下载数据"""
    base = "https://api.binance.com"
    endpoint = "/api/v3/klines"
    url = base + endpoint

    # 开始和结束时间
    start_dt = datetime.strptime(start, "%Y%m%d")
    start_dt = CHINA_TZ.localize(start_dt)

    end_dt = datetime.strptime(end, "%Y%m%d")
    end_dt = CHINA_TZ.localize(end_dt)

    # 查询缓存变量
    bar_data = {}  # 使用字典对时间戳去重
    finished = False  # 查询结束

    # 持续循环
    while True:
        # 执行REST数据查询
        params = {
            "symbol": symbol,
            "interval": "1m",
            "startTime": int(start_dt.timestamp() * 1000),
            "limit": 1000
        }
        r = requests.get(url, params=params)
        data = r.json()

        # 如果有返回的数据，则进行处理
        if data:
            for l in data:
                # 生成时间戳
                dt = datetime.fromtimestamp(l[0]/1000)
                dt = CHINA_TZ.localize(dt)

                # 检查是否已经超出结束时间，若超出则说明已经完成
                if dt > end_dt:
                    finished = True  # 标识完成状态
                    break             # 退出for循环

                # 解析K线数据
                bar = BarData(
                    symbol=symbol,
                    exchange=Exchange.BINANCE,
                    datetime=dt,
                    interval=Interval.MINUTE,
                    open_price=float(l[1]),
                    high_price=float(l[2]),
                    low_price=float(l[3]),
                    close_price=float(l[4]),
                    volume=float(l[5]),
                    gateway_name="BINANCE"
                )
                bar_data[bar.datetime] = bar

                            # 打印本轮查询范围
            print(start_dt, bar.datetime)

            # 将本轮的结束时间戳，作为新一轮的开始时间
            start_dt = bar.datetime
        # 否则说明已经结束
        else:
            finished = True

        # 结束则退出while循环
        if finished:
            break

    # 对时间戳排序
    dts = list(bar_data.keys())
    dts.sort()

    # 然后以列表形式返回
    return [bar_data[dt] for dt in dts]

In [3]:
bar_data = download_binance_minute_data("BTCUSDT", "20250801", "20250810")
print(bar_data[0])
print(bar_data[-1])

2025-08-01 00:00:00+08:00 2025-08-01 16:39:00+08:00
2025-08-01 16:39:00+08:00 2025-08-02 09:18:00+08:00
2025-08-02 09:18:00+08:00 2025-08-03 01:57:00+08:00
2025-08-03 01:57:00+08:00 2025-08-03 18:36:00+08:00
2025-08-03 18:36:00+08:00 2025-08-04 11:15:00+08:00
2025-08-04 11:15:00+08:00 2025-08-05 03:54:00+08:00
2025-08-05 03:54:00+08:00 2025-08-05 20:33:00+08:00
2025-08-05 20:33:00+08:00 2025-08-06 13:12:00+08:00
2025-08-06 13:12:00+08:00 2025-08-07 05:51:00+08:00
2025-08-07 05:51:00+08:00 2025-08-07 22:30:00+08:00
2025-08-07 22:30:00+08:00 2025-08-08 15:09:00+08:00
2025-08-08 15:09:00+08:00 2025-08-09 07:48:00+08:00
2025-08-09 07:48:00+08:00 2025-08-10 00:00:00+08:00
BarData(gateway_name='BINANCE', extra=None, symbol='BTCUSDT', exchange=<Exchange.BINANCE: 'BINANCE'>, datetime=datetime.datetime(2025, 8, 1, 0, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), interval=<Interval.MINUTE: '1m'>, volume=21.42222, turnover=0, open_interest=0, open_price=118306.18, high_price=118351.0, l

In [4]:
import pandas as pd
df = pd.DataFrame.from_records([{"close": bar.close_price} for bar in bar_data])
df.to_csv("BTC_close.csv")

In [5]:
df

,close
0,118351.00
1,118312.14
2,118312.14
3,118327.16
4,118303.49
...,...
12956,116851.99
12957,116836.72
12958,116858.81
12959,116892.47
